In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from cardinality_estimation.featurizer import Featurizer
from query_representation.query import load_qrep
from cardinality_estimation.dataset import *
from torch.utils import data

import glob
import random
import os
import json
import time
import matplotlib.pyplot as plt

# Setup file paths / Download query data

In [ ]:
import errno
def make_dir(directory):
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

In [ ]:
# TODO
# TRAINDIR = os.path.join(os.path.join("", "queries"), "mlsys1-train")
# VALDIR = os.path.join(os.path.join("", "queries"), "mlsys1-val")
# TESTDIR = os.path.join(os.path.join("", "queries"), "mlsys1-test")

TRAINDIR = os.path.join(os.path.join("", "queries"), "imdb")
TESTDIR = os.path.join(os.path.join("", "queries"), "imdb")

RESULTDIR = os.path.join("", "results")
make_dir(RESULTDIR)

# Query loading helper functions

In [ ]:

def load_qdata(fns):
    qreps = []
    for qfn in fns:
        qrep = load_qrep(qfn)
        # TODO: can do checks like no queries with zero cardinalities etc.
        qreps.append(qrep)
        template_name = os.path.basename(os.path.dirname(qfn))
        qrep["name"] = os.path.basename(qfn)
        qrep["template_name"] = template_name
    return qreps

def get_query_fns(basedir, template_fraction=1.0, sel_templates=None):
    fns = []
    tmpnames = list(glob.glob(os.path.join(basedir, "*")))
    assert template_fraction <= 1.0
    
    for qi,qdir in enumerate(tmpnames):
        if os.path.isfile(qdir):
            continue
        template_name = os.path.basename(qdir)
        if sel_templates is not None and template_name not in sel_templates:
            continue
        
        # let's first select all the qfns we are going to load
        qfns = list(glob.glob(os.path.join(qdir, "*.pkl")))
        qfns.sort()
        num_samples = max(int(len(qfns)*template_fraction), 1)
        random.seed(1234)
        qfns = random.sample(qfns, num_samples)
        fns += qfns
    return fns

# Evaluation helper functions

In [ ]:
def get_preds(alg, qreps):
    if isinstance(qreps[0], str):
        # only file paths sent
        qreps = load_qdata(qreps)
    
    ests = alg.test(qreps)
    return ests

def eval_alg(alg, eval_funcs, qreps, samples_type, result_dir="./results/"):
    '''
    '''
    np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

    alg_name = alg.__str__()
    exp_name = alg.get_exp_name()
    
    if isinstance(qreps[0], str):
        # only file paths sent
        qreps = load_qdata(qreps)
    
    ests = alg.test(qreps)

    for efunc in eval_funcs:
        rdir = None
        if result_dir is not None:
            rdir = os.path.join(result_dir, exp_name)
            make_dir(rdir)

        errors = efunc.eval(qreps, ests, samples_type=samples_type,
                result_dir=rdir,
                num_processes = -1,
                alg_name = alg_name,
                use_wandb=0)

        print("{}, {}, #samples: {}, {}: mean: {}, median: {}, 99p: {}"\
                .format(samples_type, alg, len(errors),
                    efunc.__str__(),
                    np.round(np.mean(errors),3),
                    np.round(np.median(errors),3),
                    np.round(np.percentile(errors,99),3)))

# Load queries

In [ ]:
# set template_fraction <= 1.0 to test quickly w/ smaller datasets
# train_qfns = get_query_fns(TRAINDIR, template_fraction = 0.001)
# val_qfns = get_query_fns(VALDIR, template_fraction = 1.0)
# test_qfns = get_query_fns(TESTDIR, template_fraction = 1.0)

train_qfns = get_query_fns(TRAINDIR, template_fraction = 1.0, sel_templates=["2b"])
val_qfns = []
test_qfns = get_query_fns(TESTDIR, template_fraction = 1.0, sel_templates=["2a"])

print("Selected {} training queries, {} validation queries, {} test queries".\
      format(len(train_qfns), len(val_qfns), len(test_qfns)))

In [ ]:
from evaluation.eval_fns import QError, SimplePlanCost
EVAL_FNS = []
EVAL_FNS.append(QError())
EVAL_FNS.append(SimplePlanCost())

In [ ]:
def init_featurizer(featurization_type):
    # Load database specific data, e.g., information about columns, tables etc.
    dbdata_fn = os.path.join(TRAINDIR, "dbdata.json")
    featurizer = Featurizer(None, None, None, None, None)
    with open(dbdata_fn, "r") as f:
        dbdata = json.load(f)
    featurizer.update_using_saved_stats(dbdata)

    featurizer.setup(ynormalization="log",
        feat_separate_alias = 0,
        onehot_dropout = onehot_dropout,
        feat_mcvs = 0,
        heuristic_features = 1,
        featurization_type=featurization_type,
        table_features=1,
        flow_features = 0,
        join_features= "onehot",
        set_column_feature= "onehot",
        max_discrete_featurizing_buckets=30,
        max_like_featurizing_buckets=10,
        embedding_fn = "none",
        embedding_pooling = None,
        implied_pred_features = 0,
        feat_onlyseen_preds = 1)
    featurizer.update_ystats(trainqs)
    
    featurizer.update_workload_stats(trainqs)
    featurizer.init_feature_mapping()
    featurizer.update_ystats(trainqs)

    # if feat_onlyseen_preds:
    # just do it always
    featurizer.update_seen_preds(trainqs)
    
    return featurizer

In [ ]:
# going to start training the models
trainqs = load_qdata(train_qfns)

In [ ]:
testqs = load_qdata(test_qfns)

In [ ]:
max_epochs = 60
lr=0.0001
training_opt = "none"
opt_lr = 0.1
swa_start = 5
mask_unseen_subplans = 0
subplan_level_outputs=0
normalize_flow_loss = 1
heuristic_unseen_preds = 0
cost_model = "C"
use_wandb = 0
eval_fns = "qerr,plancost"
load_padded_mscn_feats = 1
mb_size = 1024
weight_decay = 0.0
load_query_together = 0
result_dir = "./results"
onehot_dropout=0
onehot_mask_truep=0.8
onehot_reg=0
onehot_reg_decay=0.1
eval_epoch = 200
optimizer_name="adamw"
clip_gradient=20.0
loss_func_name = "mse"
hidden_layer_size = 128
num_hidden_layers = 2

In [ ]:
# from cardinality_estimation.fcnn import FCNN
# featurizer = init_featurizer("combined")
# #featurizer = init_featurizer("set")

# fcnn = FCNN(max_epochs = max_epochs, lr=lr,
#                 training_opt = training_opt,
#                 opt_lr = opt_lr,
#                 swa_start = swa_start,
#                 mask_unseen_subplans = mask_unseen_subplans,
#                 subplan_level_outputs=subplan_level_outputs,
#                 normalize_flow_loss = normalize_flow_loss,
#                 heuristic_unseen_preds = heuristic_unseen_preds,
#                 onehot_dropout=onehot_dropout,
#                 onehot_reg=onehot_reg,
#                 onehot_reg_decay=onehot_reg_decay,
#                 cost_model = cost_model,
#                 eval_fns = eval_fns,
#                 use_wandb = use_wandb,
#                 mb_size = mb_size,
#                 weight_decay = weight_decay,
#                 load_query_together = load_query_together,
#                 result_dir = result_dir,
#                 num_hidden_layers=num_hidden_layers,
#                 eval_epoch = eval_epoch,
#                 optimizer_name=optimizer_name,
#                 clip_gradient=clip_gradient,
#                 loss_func_name = loss_func_name,
#                 hidden_layer_size = hidden_layer_size)

# fcnn.train(trainqs, valqs=None, testqs=None,
#     featurizer=featurizer, result_dir=RESULTDIR)

In [ ]:
# # # evaluate model
# eval_alg(fcnn, EVAL_FNS, trainqs, "train")
# # eval_alg(fcnn, EVAL_FNS, valqs, "val")

# # # TODO: test set prdictions; should submit these for the leaderboard?
# # #preds = fcnn.test(testqs)

In [ ]:
from cardinality_estimation.mscn import MSCN as MSCN2

featurizer = init_featurizer("set")
mscn = MSCN2(max_epochs = max_epochs, lr=lr,
                training_opt = training_opt,
                inp_dropout = 0.0,
                hl_dropout = 0.0,
                comb_dropout = 0.0,
                opt_lr = opt_lr,
                swa_start = swa_start,
                mask_unseen_subplans = mask_unseen_subplans,
                subplan_level_outputs=subplan_level_outputs,
                normalize_flow_loss = normalize_flow_loss,
                heuristic_unseen_preds = heuristic_unseen_preds,
                cost_model = cost_model,
                use_wandb = use_wandb,
                eval_fns = eval_fns,
                load_padded_mscn_feats = load_padded_mscn_feats,
                mb_size = mb_size,
                weight_decay = weight_decay,
                load_query_together = load_query_together,
                result_dir = result_dir,
                onehot_dropout=onehot_dropout,
                onehot_mask_truep=onehot_mask_truep,
                onehot_reg=onehot_reg,
                onehot_reg_decay=onehot_reg_decay,
                # num_hidden_layers=num_hidden_layers,
                eval_epoch = eval_epoch,
                optimizer_name=optimizer_name,
                clip_gradient=clip_gradient,
                loss_func_name = loss_func_name,
                hidden_layer_size = hidden_layer_size)

mscn.train(trainqs, valqs=None, testqs=None,
    featurizer=featurizer, result_dir=RESULTDIR)

In [ ]:
# evaluate model
eval_alg(mscn, EVAL_FNS, trainqs, "train")

#eval_alg(mscn, EVAL_FNS, valqs, "val")
# TODO: test set prdictions; should submit these for the leaderboard?
#preds = mscn.test(testqs)

In [ ]:
eval_alg(mscn, EVAL_FNS, testqs, "test")

In [ ]:
# imports from captum library
from captum.attr import LayerConductance, LayerActivation, LayerIntegratedGradients
from captum.attr import IntegratedGradients, DeepLift, GradientShap, NoiseTunnel, FeatureAblation

In [ ]:
featurizer = init_featurizer("set")
ds = QueryDataset(trainqs[0:10], featurizer,
        True,
        load_padded_mscn_feats=True)
loader = data.DataLoader(ds,
        batch_size=1, shuffle=False,
        collate_fn=mscn_collate_fn_together,
        )

testds = QueryDataset(testqs[0:3], featurizer,
        True,
        load_padded_mscn_feats=True)
testloader = data.DataLoader(testds,
        batch_size=1, shuffle=False,
        collate_fn=mscn_collate_fn_together,
        )

In [ ]:
import sqlparse
sql = trainqs[2]["sql"]
print(sqlparse.format(sql, reindent=True, keyword_case='upper'))

In [ ]:
#iloader = iter(loader)
iloader = iter(testloader)

In [ ]:
xbatch,y,info = next(iloader)
print(torch.sum(xbatch["pred"][:,:,20]))
print(torch.sum(xbatch["pred"][:,:,21]))

In [ ]:
#model.state_dict()

In [ ]:
from cardinality_estimation.nets import *
mscn_model = mscn.net
n_out = 1
sfeats = mscn_model.sample_mlp1.in_features
pfeats = mscn_model.predicate_mlp1.in_features
jfeats = mscn_model.join_mlp1.in_features

net = SetConvNoFlow(sfeats,
    pfeats, jfeats,
    128,
    n_out=n_out,
    dropouts=[0.0, 0.0, 0.0])

In [ ]:
print(net)
print(mscn_model)
net.load_state_dict(mscn_model.state_dict())

In [ ]:
torch.sum(xbatch["pred"][:,:,20])

In [ ]:
model = net
model.eval()
ig = IntegratedGradients(model)
ig_nt = NoiseTunnel(ig)
dl = DeepLift(model)
gs = GradientShap(model)
fa = FeatureAblation(model)

In [ ]:
#xbatch = ds.X[32]
# ig_attr_test = ig.attribute(tuple([xbatch["table"].unsqueeze(0), xbatch["pred"].unsqueeze(0),
#                             xbatch["join"].unsqueeze(0), xbatch["flow"].unsqueeze(0),
#                             xbatch["tmask"].unsqueeze(0), xbatch["pmask"].unsqueeze(0), 
#                                    xbatch["jmask"].unsqueeze(0)]), n_steps=50)
ig_attr_test = ig.attribute(tuple([xbatch["table"], xbatch["pred"],
                            xbatch["join"], 
                            xbatch["tmask"], xbatch["pmask"], 
                                   xbatch["jmask"]]), n_steps=50)
print("ig done")

fa_attr_test = fa.attribute(tuple([xbatch["table"], xbatch["pred"],
                            xbatch["join"],
                            xbatch["tmask"], xbatch["pmask"], 
                                   xbatch["jmask"]]), n_steps=50)
print("fa done")

In [ ]:
# def get_attr_vecs(curx, curattrs):
#     #idxs = 0
#     xsum = curx.sum(axis=0).sum(axis=0)
#     zero_idxs = xsum == 0
#     curattrs = curattrs[:,:,~zero_idxs]
#     idxs = np.array(range(len(xsum)))[~zero_idxs]
#     curx = curx[:,:,~zero_idxs]
    
#     assert curx.shape == curattrs.shape
    
#     # TODO: avg based on non-zero elements
#     attr_sum = curattrs.sum(axis=0).sum(axis=0)
#     assert attr_sum.shape[0] == curx.shape[-1]
    
#     # TODO: do we need this?
#     #attr_sum = attr_sum / np.linalg.norm(attr_sum, ord=1)
    
#     # TODO: do this or no?
#     curx_nonz = curx != 0
#     xnonzero_sums = curx_nonz.sum(axis=0).sum(axis=0)
#     #xnonzero_sums += 1
#     #print(xnonzero_sums)
#     #print(xnonzero_sums)
#     attr_sum = attr_sum / xnonzero_sums
    
#     return idxs, attr_sum

# def get_mscn_attrs(xbatch, attrs, featurizer, normalize=True):
#     '''
#     returns a vector with x-axis names and attribution values;
#     '''
#     batchsize = xbatch["table"].shape[0]
#     assert batchsize == attrs[0].shape[0]
#     tabidxs, tabattrs = get_attr_vecs(xbatch["table"].detach().numpy(), attrs[0].detach().numpy())
#     predidxs, predattrs = get_attr_vecs(xbatch["pred"].detach().numpy(), attrs[1].detach().numpy())
#     joinidxs, joinattrs = get_attr_vecs(xbatch["join"].detach().numpy(), attrs[2].detach().numpy())
    
#     # TODO: need to do sample_bitmaps
#     tablabels = []
#     for curtabidx in tabidxs:
#         for tab,tidx in featurizer.table_featurizer.items():
#             if tidx == curtabidx:
#                 tablabels.append(tab)
#                 break
#     joinlabels = []
#     for curjidx in joinidxs:
#         for join,jidx in featurizer.join_featurizer.items():
#             found = False
#             if curjidx == jidx:
#                 joinlabels.append(join)
#                 found = True
#                 break
#         if not found:
#             joinlabels.append("unknown")
#     # TODO: join-stats
    
#     predlabels = []
#     colstart,collen = featurizer.featurizer_type_idxs["col_onehot"]
#     # TODO: if stats used
#     #colstatsstart,colstatsend = self.featurizer_type_idxs["col_stats"]
#     cmp_start,cmplen = featurizer.featurizer_type_idxs["cmp_op"]
#     cstart,clen = featurizer.featurizer_type_idxs["constant_continuous"]
#     lstart,llen = featurizer.featurizer_type_idxs["constant_like"]
#     dstart,dlen = featurizer.featurizer_type_idxs["constant_discrete"]
#     hstart,hlen = featurizer.featurizer_type_idxs["heuristic_ests"]
#     #print(hstart, hlen)
#     for pi in predidxs:
#         if pi >= colstart and pi < colstart+collen:
#             found = False
#             for col,colidx in featurizer.columns_onehot_idx.items():
#                 if colidx == pi:
#                     predlabels.append(col)
#                     found = True
#                     break
#             if not found:
#                 print(pi)
#                 predlabels.append("col-unknown")
#         elif pi >= cmp_start and pi < cmp_start+cmplen:
#             predlabels.append("cmp")
#         elif pi == cstart:
#             predlabels.append("<")
#         elif pi == cstart+1:
#             predlabels.append(">")
#         elif pi >= lstart and pi < lstart+llen:
#             predlabels.append("Like-Hash-" + str(pi))
#         elif pi >= dstart and pi < dstart+dlen:
#             predlabels.append("Constant-Hash-" + str(pi))
#         elif pi == hstart:
#             predlabels.append("PostgreSQL Est (Table)")
#         elif pi == hstart+1:
#             predlabels.append("PostgreSQL Est (Subplan)")
    
#     #assert len(predidxs) == len(predlabels)
# #     print(len(predidxs), len(predlabels))
# #     print(predidxs)
# #     print(predlabels)
#     attrs = np.concatenate([tabattrs, joinattrs, predattrs])
#     xlabels = tablabels + joinlabels + predlabels
    
#     if normalize:
#         attrs = attrs / np.linalg.norm(attrs, ord=1)
    
#     #xlabels = list(range(len(attrs)))
#     return xlabels,attrs

In [ ]:
def get_attr_vecs(curx, curattrs):
    #idxs = 0
    xsum = curx.sum(axis=0).sum(axis=0)
    zero_idxs = xsum == 0
    curattrs = curattrs[:,:,~zero_idxs]
    idxs = np.array(range(len(xsum)))[~zero_idxs]
    curx = curx[:,:,~zero_idxs]
    
    assert curx.shape == curattrs.shape
    
    # TODO: avg based on non-zero elements
    
    print("attr sum: ", np.sum(curattrs))
    curattrs = np.abs(curattrs)
    print("attr sum after abs: ", np.sum(curattrs))
    
    attr_sum = curattrs.sum(axis=0).sum(axis=0)
    
    assert attr_sum.shape[0] == curx.shape[-1]
    
    # TODO: do we need this?
    #attr_sum = attr_sum / np.linalg.norm(attr_sum, ord=1)
    
    # TODO: do this or no?
    curx_nonz = curx != 0
    
    xnonzero_sums = curx_nonz.sum(axis=0).sum(axis=0)
    
    #TODO?
    attr_sum = attr_sum / xnonzero_sums
    
    return idxs, attr_sum

def get_mscn_attrs(xbatch, attrs, featurizer, normalize=True):
    '''
    returns a vector with x-axis names and attribution values;
    '''
    batchsize = xbatch["table"].shape[0]
    assert batchsize == attrs[0].shape[0]
    tabidxs, tabattrs = get_attr_vecs(xbatch["table"].detach().numpy(), attrs[0].detach().numpy())
    predidxs, predattrs = get_attr_vecs(xbatch["pred"].detach().numpy(), attrs[1].detach().numpy())
    joinidxs, joinattrs = get_attr_vecs(xbatch["join"].detach().numpy(), attrs[2].detach().numpy())
    
    # TODO: need to do sample_bitmaps
    tablabels = []
    for curtabidx in tabidxs:
        for tab,tidx in featurizer.table_featurizer.items():
            if tidx == curtabidx:
                tablabels.append(tab)
                break
    joinlabels = []
    for curjidx in joinidxs:
        for join,jidx in featurizer.join_featurizer.items():
            found = False
            if curjidx == jidx:
                joinlabels.append(join)
                found = True
                break
        if not found:
            joinlabels.append("unknown")
    # TODO: join-stats
    
    predlabels = []
    colstart,collen = featurizer.featurizer_type_idxs["col_onehot"]
    # TODO: if stats used
    #colstatsstart,colstatsend = self.featurizer_type_idxs["col_stats"]
    cmp_start,cmplen = featurizer.featurizer_type_idxs["cmp_op"]
    cstart,clen = featurizer.featurizer_type_idxs["constant_continuous"]
    lstart,llen = featurizer.featurizer_type_idxs["constant_like"]
    dstart,dlen = featurizer.featurizer_type_idxs["constant_discrete"]
    hstart,hlen = featurizer.featurizer_type_idxs["heuristic_ests"]
    #print(hstart, hlen)
    for pi in predidxs:
        if pi >= colstart and pi < colstart+collen:
            found = False
            for col,colidx in featurizer.columns_onehot_idx.items():
                if colidx == pi:
                    predlabels.append(col)
                    found = True
                    break
            if not found:
                print(pi)
                predlabels.append("col-unknown")
        elif pi >= cmp_start and pi < cmp_start+cmplen:
            predlabels.append("cmp")
        elif pi == cstart:
            predlabels.append("<")
        elif pi == cstart+1:
            predlabels.append(">")
        elif pi >= lstart and pi < lstart+llen:
            predlabels.append("Like-Hash-" + str(pi))
        elif pi >= dstart and pi < dstart+dlen:
            predlabels.append("Constant-Hash-" + str(pi))
        elif pi == hstart:
            predlabels.append("PostgreSQL Est (Table)")
        elif pi == hstart+1:
            predlabels.append("PostgreSQL Est (Subplan)")
    
    #assert len(predidxs) == len(predlabels)
#     print(len(predidxs), len(predlabels))
#     print(predidxs)
#     print(predlabels)
    attrs = np.concatenate([tabattrs, joinattrs, predattrs])
    xlabels = tablabels + joinlabels + predlabels
    
    if normalize:
        attrs = attrs / np.linalg.norm(attrs, ord=1)
    
    return xlabels,attrs

In [ ]:
xlabels, igattrs = get_mscn_attrs(xbatch, ig_attr_test, featurizer, normalize=True)
xlabels, fattrs = get_mscn_attrs(xbatch, fa_attr_test, featurizer, normalize=True)
print(xlabels)
print(igattrs)

In [ ]:
import seaborn as sns

# plt.figure(figsize=(30, 10))
# sns.barplot(x=xlabels, y=igattrs, color='#4260f5')

plt.figure(figsize=(30, 20))
plt.yticks(fontsize=20)
sns.barplot(x=igattrs, y=xlabels, color='#4260f5', orient="horizontal")
plt.savefig("captum-mscn-dropout-2b.png")

In [ ]:
import seaborn as sns

plt.figure(figsize=(30, 10))
sns.barplot(x=xlabels, y=fattrs, color='#4260f5')

In [ ]:
curattrs = ig_attr_test[0].detach().numpy()
print(curattrs.shape)
curattrs.sum(axis=0).sum(axis=0).shape

In [ ]:
featurizer.featurizer_type_idxs

In [ ]:
print(xbatch["table"].sum(axis=[0,1]))
print(xbatch["pred"].sum(axis=[0,1]))
print(xbatch["join"].sum(axis=[0,1]))

print(xbatch["table"].shape)
print(xbatch["pred"].shape)
print(xbatch["join"].shape)

In [ ]:
# import torch
# print(torch.max(ig_attr_test[1]), torch.argmax(ig_attr_test[1]))
# print(torch.max(fa_attr_test[1]), torch.argmax(fa_attr_test[1]))
#len(ig_attr_test)

# np.linalg.norm(ig_attr_test[1].detach().numpy(), ord=0)

In [ ]:
ig_attr_test[1].shape

In [ ]:
torch.sum(ig_attr_test[1][:,:,idx])

In [ ]:
for idx in range(ig_attr_test[1].shape[2]):
    print(idx, torch.sum(ig_attr_test[1][:,:,idx]))

In [ ]:
featurizer = init_featurizer("combined")
ds = QueryDataset(trainqs, featurizer,
        False,
        load_padded_mscn_feats=False)

# featurizer = init_featurizer("set")
# ds = QueryDataset(trainqs, featurizer,
#         False,
#         load_padded_mscn_feats=True)
model = fcnn.net

ig = IntegratedGradients(model)
ig_nt = NoiseTunnel(ig)
dl = DeepLift(model)
gs = GradientShap(model)
fa = FeatureAblation(model)

In [ ]:
X_test = ds.X[32:35]
X_train = ds.X[32:35]

ig_attr_test = ig.attribute(X_test, n_steps=50)
print("ig done")
ig_nt_attr_test = ig_nt.attribute(X_test)
dl_attr_test = dl.attribute(X_test)
print("dl done")
gs_attr_test = gs.attribute(X_test, X_train)
print("gs done")
fa_attr_test = fa.attribute(X_test)

In [ ]:
ig_attr_test

In [ ]:
X_test.shape

In [ ]:
# lets find all the zeros in X_test
combined_vec = X_test.sum(axis=0)
zero_idxs = combined_vec == 0
X_test_nonzero = X_test[:,~zero_idxs]
ig_attr_test = ig_attr_test[:,~zero_idxs]
ig_nt_attr_test = ig_nt_attr_test[:,~zero_idxs]
dl_attr_test = dl_attr_test[:,~zero_idxs]
gs_attr_test = gs_attr_test[:,~zero_idxs]
fa_attr_test = fa_attr_test[:,~zero_idxs]

In [ ]:
model = fcnn.net

In [ ]:
x_axis_data = np.arange(X_test_nonzero.shape[1])
x_axis_data_labels = list(map(lambda idx: idx, x_axis_data))

ig_attr_test_sum = ig_attr_test.detach().numpy().sum(0)
ig_attr_test_norm_sum = ig_attr_test_sum / np.linalg.norm(ig_attr_test_sum, ord=1)

ig_nt_attr_test_sum = ig_nt_attr_test.detach().numpy().sum(0)
ig_nt_attr_test_norm_sum = ig_nt_attr_test_sum / np.linalg.norm(ig_nt_attr_test_sum, ord=1)

dl_attr_test_sum = dl_attr_test.detach().numpy().sum(0)
dl_attr_test_norm_sum = dl_attr_test_sum / np.linalg.norm(dl_attr_test_sum, ord=1)

gs_attr_test_sum = gs_attr_test.detach().numpy().sum(0)
gs_attr_test_norm_sum = gs_attr_test_sum / np.linalg.norm(gs_attr_test_sum, ord=1)

fa_attr_test_sum = fa_attr_test.detach().numpy().sum(0)
fa_attr_test_norm_sum = fa_attr_test_sum / np.linalg.norm(fa_attr_test_sum, ord=1)

lin_weight = model.layers[0][0].weight.detach().numpy()
y_axis_lin_weight = lin_weight / np.linalg.norm(lin_weight, ord=1)

width = 0.14
legends = ['Int Grads', 'Int Grads w/SmoothGrad','DeepLift', 'GradientSHAP', 'Feature Ablation', 'Weights']

plt.figure(figsize=(30, 10))

ax = plt.subplot()
ax.set_title('Comparing input feature importances across multiple algorithms and learned weights')
ax.set_ylabel('Attributions')

FONT_SIZE = 16
plt.rc('font', size=FONT_SIZE)            # fontsize of the text sizes
plt.rc('axes', titlesize=FONT_SIZE)       # fontsize of the axes title
plt.rc('axes', labelsize=FONT_SIZE)       # fontsize of the x and y labels
plt.rc('legend', fontsize=FONT_SIZE - 4)  # fontsize of the legend

ax.bar(x_axis_data, ig_attr_test_norm_sum, width, align='center', alpha=0.8, color='#eb5e7c')
ax.bar(x_axis_data + width, ig_nt_attr_test_norm_sum, width, align='center', alpha=0.7, color='#A90000')
ax.bar(x_axis_data + 2 * width, dl_attr_test_norm_sum, width, align='center', alpha=0.6, color='#34b8e0')
ax.bar(x_axis_data + 3 * width, gs_attr_test_norm_sum, width, align='center',  alpha=0.8, color='#4260f5')
ax.bar(x_axis_data + 4 * width, fa_attr_test_norm_sum, width, align='center', alpha=1.0, color='#49ba81')

#ax.bar(x_axis_data + 5 * width, y_axis_lin_weight, width, align='center', alpha=1.0, color='grey')
ax.autoscale_view()
plt.tight_layout()

ax.set_xticks(x_axis_data + 0.5)
ax.set_xticklabels(x_axis_data_labels)

plt.legend(legends, loc=3)
plt.show()

In [ ]:
np.linalg.norm(dl_attr_test_sum, ord=1)

In [ ]:
dl_attr_test_sum

In [ ]:
dl_attr_test.sum(0)